In [1]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-70b-versatile", temperature=0)

model.invoke("Hello, world!")


AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 39, 'total_tokens': 49, 'completion_time': 0.04, 'prompt_time': 0.006590195, 'queue_time': 0.008387084, 'total_time': 0.046590195}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b6828be2c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-b44915b5-dbfb-44e6-89d8-986aa98cc77e-0', usage_metadata={'input_tokens': 39, 'output_tokens': 10, 'total_tokens': 49})

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Create a masters dissertation draft according to the requirement from the human"),
    HumanMessage("jet noise interaction with the wing. experimental, semi-empirical, analytical and simulation approaches.")
]

for token in model.stream(messages):
    print(token.content, end="")


**Master's Dissertation Draft:**

**Title:** Investigation of Jet Noise Interaction with the Wing: Experimental, Semi-Empirical, Analytical, and Simulation Approaches

**Abstract:**

The interaction between jet noise and wing is a complex phenomenon that affects the overall noise levels and aerodynamic performance of aircraft. This dissertation aims to investigate the jet noise interaction with the wing using experimental, semi-empirical, analytical, and simulation approaches. The study focuses on understanding the underlying mechanisms of jet noise generation, propagation, and interaction with the wing. The results of this research will contribute to the development of more accurate noise prediction models and optimization of wing design for reduced noise levels.

**Introduction:**

Jet noise is a significant contributor to the overall noise levels of aircraft, particularly during takeoff and landing. The interaction between jet noise and the wing can lead to increased noise levels, r

## Você pode criar templates de prompt, com placeholders para substituir automaticamente

In [3]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following sentence from English to {language}"
user_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", user_template)]
)

In [9]:
prompt = prompt_template.invoke({"language": "brazilian portuguese", "text": "hey guys! how are you all doing?"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following sentence from English to brazilian portuguese', additional_kwargs={}, response_metadata={}), HumanMessage(content='hey guys! how are you all doing?', additional_kwargs={}, response_metadata={})])

In [6]:
prompt.to_messages()

[SystemMessage(content='Translate the following sentence from English to italian', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hey guys! how are you all doing?', additional_kwargs={}, response_metadata={})]

In [11]:
response = model.invoke(prompt)
print(response.content)

Ei pessoal! Como vocês estão?


# Output estruturado, permitindo o modelo escolher a estrutura mais adequada ao prompt

In [65]:
from typing import Optional, Union

from pydantic import BaseModel, Field
from enum import Enum

# Pydantic
class Joke(BaseModel):
    """Joke to tell the user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: int = Field(
        default=-1, description="How funny the joke is, from 1 to 10"
    )

class VoiceOptions(Enum):
    A = "creepy"
    B = "gentle"
    C = "misterious"
    D = "sad"

class Story(BaseModel):
    """Story to be told to the user."""

    soundtrack: str = Field(description="A specific song to be played while the story is read, to give the emotion the story evokes.")
    voice_option: VoiceOptions = Field(description="The tone that the person who reads the story must have to give the emotion the story evokes.")
    story: str = Field(description="The story to be read to the user")

class ConversationalResponse(BaseModel):
    """Respond in a conversational manner and be kind."""

    response: str = Field(description="A conversational response to the user's query.")

class AnswerOptions(BaseModel):
    answer: Union[Joke, Story, ConversationalResponse]

structured_llm = model.with_structured_output(AnswerOptions)

In [61]:
answer = structured_llm.invoke("Tell me a joke about cats")
answer

AnswerOptions(answer=Joke(setup='Why did the cat join a band?', punchline='Because it wanted to be the purr-cussionist!', rating=8))

In [66]:
answer = structured_llm.invoke("Tell me story about the sad life of a lonely ant")
answer

AnswerOptions(answer=Story(soundtrack='Mad World by Gary Jules', voice_option=<VoiceOptions.D: 'sad'>, story='Once upon a time, in a bustling colony of ants, there was one ant named Andy. Andy lived a life of solitude, often finding himself lost in the crowd of his own kin. While his fellow ants worked tirelessly to gather food and build their home, Andy struggled to find his place. He felt like an outcast, a small, insignificant speck in the grand scheme of things. As the days went by, Andy became increasingly withdrawn, spending most of his time alone, wandering the desolate outskirts of the colony. He longed for connection, for someone to share his thoughts and feelings with, but it seemed like no one was interested. The other ants were too busy with their own lives to notice Andy’s struggles. As the seasons changed, Andy’s loneliness only deepened. He felt like he was drowning in a sea of faces, unable to find a lifeline to cling to. His heart ached with a sadness that seemed to ha

In [67]:
answer.answer.voice_option.value

'sad'